In [ ]:
%matplotlib inline
# Basic numerical package
import numpy as np
import pandas as pd
from astropy.io import fits

# Graphical packages
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix
from astropy.table import Table

# Package for unsupervised learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import umap
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn import metrics
from sklearn.cluster import OPTICS

# warning
import warnings

warnings.filterwarnings("ignore")

# some settings for the notebook
plt.rcParams.update({"font.size": 12})
pd.set_option("display.max_columns", 600)
sns.set_theme(style="whitegrid", font_scale=1.5, context="paper")


In [ ]:
# Fancy figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Data Inspection and Cleaning

In [ ]:
#Importing the relevant data
sdss = Table.read('SDSS17Pipe3D_v3_1_1.fits')
df = sdss.to_pandas()
df
# df.columns #To view the table

In [ ]:
#Printing column names
for col in df.columns:
    print("'" + col + "',")
#column = np.loadtxt("columns.txt", dtype='str')

#Keeping relevant columns
col = ['R50_kpc_V', 'R50_kpc_Mass', 'u_band_mag', 'g_band_mag', 'r_band_mag', 'B_band_mag', 'V_band_mag', 'B-V', 'B-R','u-g', 'g-r', 'r-i', 'i-z', 'C', 'R90','R50',
       'Re_kpc', 'vel_sigma_Re', 'RJ_band_mag', 'best_type_n']

In [ ]:
#Cleaning the data

#Dropping nans
df = df[col]
df = df.dropna()

#Checking for duplicates 
duplicates = df.duplicated()
for i in duplicates:
    if i==True:
        print('Duplicate rows:',i)
print(df[duplicates])

In [ ]:
#Looking at column counts, mean, std, etc...
df.describe()

In [ ]:
#Creating a scatter matrix of interested features
#for relationshiop overview
test = df.iloc[:, -1]
features = df.iloc[:, :-1]

matrix = scatter_matrix(features, figsize=(16, 16), diagonal='kde')

#Rotating the labels of the data columns
for ax in matrix.ravel():
    ax.set_xlabel(ax.get_xlabel(), rotation=45)
    ax.set_ylabel(ax.get_ylabel(), rotation=60)

# plt.tight_layout()
plt.show()

In [ ]:
#Scaling the data for easier manipulation
df_test = df.iloc[:, -1]
df_features = df.iloc[:, :-1]

scaler = StandardScaler()
scaler.fit(df_features)

scaled_data = scaler.transform(df_features)
print("Scaled Data:\n", scaled_data)

#Converting back to df
df_scaled = pd.DataFrame(scaled_data, columns=df_features.columns)

print(df_scaled)

In [ ]:
#Plotting a correlation matrix to check on feature correlations
df_mat = df_scaled.apply(pd.to_numeric, errors='coerce')
matrix = df_mat.corr(min_periods=1)
plt.imshow(matrix, cmap='Blues')
plt.colorbar()

#Extracting variable names 
variables = []
for i in matrix.columns:
    variables.append(i)

plt.xticks(range(len(matrix)), variables, rotation=45, ha='right')
plt.yticks(range(len(matrix)), variables)
plt.show()

### Dimensionality Reduction

In [ ]:
#PCA DR
pca = PCA(n_components=3, svd_solver='auto')
X_pca = pca.fit_transform(df_scaled)
print(pca.get_feature_names_out())

#Plotting the results in 3D
fig,ax = plt.subplots(1,1, figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:,2], c=df_test, s = 0.5, alpha=0.5)
plt.colorbar(scatter, ax=ax)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')
plt.title('3D PCA of Galaxy Data')
plt.show()

In [ ]:
# t-SNE DR
tsne = TSNE(n_components=2, perplexity=50, learning_rate=800)
X_tsne = tsne.fit_transform(df_scaled)


#Plotting
fig,ax = plt.subplots(1,1, figsize=(10, 8))

scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1], c=df_test, s=1, alpha=0.5)
plt.colorbar(scatter, ax=ax)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('2D tSNE of Galaxy Data')

plt.show()

In [ ]:
# t-SNE DR : lower perplexity
tsne2 = TSNE(n_components=2, perplexity=20, learning_rate=800)
X_tsne2 = tsne2.fit_transform(df_scaled)


#Plotting
fig,ax = plt.subplots(1,1, figsize=(10, 8))

scatter = ax.scatter(X_tsne2[:, 0], X_tsne2[:, 1], c=df_test, s=1, alpha=0.5)
plt.colorbar(scatter, ax=ax)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('2D tSNE of Galaxy Data')

plt.show()

In [ ]:
#UMAP DR
umap_model = umap.UMAP(n_components=3)
X_umap = umap_model.fit_transform(df_scaled)

#Plotting
fig,ax = plt.subplots(1,1, figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

scatter2 = ax.scatter(X_umap[:, 0], X_umap[:, 1], X_umap[:, 2], c=df_test, cmap='viridis', s=1, alpha=0.5)
plt.colorbar(scatter2, ax=ax)
plt.title('UMAP projection of the dataset')
ax.set_xlabel('UMAP Component 1')
ax.set_ylabel('UMAP Component 2')
ax.set_zlabel('UMAP Component 3')
plt.show()

### Clustering

In [ ]:
#Kmeans on PCA clustering
kmeans = KMeans(n_clusters=10)
clusters = kmeans.fit_predict(X_pca)

#Plotting in 3D
fig,ax = plt.subplots(1,1, figsize=(10, 8))
ax = fig.add_subplot(projection='3d')
scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1],  X_pca[:,2], c=clusters, cmap='viridis', marker='o', edgecolor='k', s=10)
scatter2 = ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=10, c='red', marker='X')
plt.colorbar(scatter, ax=ax)
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')
plt.title('KMeans Clusters after PCA')
plt.show()

In [ ]:
#Kmeans on tSNE
kmeans2 = KMeans(n_clusters=10)
clusters2 = kmeans2.fit_predict(X_tsne)

#Plotting
plt.figure(figsize=(10, 6))
scat = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=clusters2, cmap='viridis', marker='o', edgecolor='k', s=10)
plt.scatter(kmeans2.cluster_centers_[:, 0], kmeans2.cluster_centers_[:, 1], s=10, c='red', marker='X')
plt.colorbar(scat)
plt.title('KMeans Clusters after t-SNE perplexity=50')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE  Component 2')
plt.show()

In [ ]:
#Kmeans on tSNE (lower perplexity)
kmeans3 = KMeans(n_clusters=10)
clusters3 = kmeans3.fit_predict(X_tsne2)

#Plotting
plt.figure(figsize=(10, 6))
scat2 = plt.scatter(X_tsne2[:, 0], X_tsne2[:, 1], c=clusters3, cmap='viridis', marker='o', edgecolor='k', s=10)
plt.scatter(kmeans3.cluster_centers_[:, 0], kmeans3.cluster_centers_[:, 1], s=10, c='red', marker='X')
plt.colorbar(scat2)
plt.title('KMeans Clusters after t-SNE perplexity=20')
plt.xlabel('t-SNE  Component 1')
plt.ylabel('t-SNE  Component 2')
plt.show()

In [ ]:
#KMeans on UMAP
kmeans4 = KMeans(n_clusters=10)
clusters4 = kmeans4.fit_predict(X_umap)

#Plotting in 3D
fig,ax = plt.subplots(1,1, figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

scatter4 = ax.scatter(X_umap[:, 0], X_umap[:, 1],  X_umap[:,2], c=clusters4, cmap='viridis', marker='o', edgecolor='k', s=10)
scatter5 = ax.scatter(kmeans4.cluster_centers_[:, 0], kmeans4.cluster_centers_[:, 1], s=10, c='red', marker='X')
plt.colorbar(scatter4, ax=ax)
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')
plt.title('KMeans Clusters after UMAP')
plt.show()

In [ ]:
#DBSCAN on PCA
dbs = DBSCAN(min_samples=2, eps=0.3)
clust = dbs.fit_predict(X_pca)

#Plotting in 3D
fig,ax = plt.subplots(1,1, figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1],  X_pca[:,2], c=clust, cmap='viridis', marker='o', edgecolor='k', s=10)
plt.colorbar(scatter, ax=ax)
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')
plt.title('DBSCAN Clusters after PCA')
plt.show()

In [ ]:
#DBSCAN on tSNE
dbs2 = DBSCAN(min_samples=2, eps=0.3)
clust2 = dbs2.fit_predict(X_tsne)

#Plotting
plt.figure(figsize=(10, 6))

scat3 = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=clust2, cmap='viridis', marker='o', edgecolor='k', s=10)
plt.colorbar(scat3)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('DBSCAN Clusters after t-SNE perplexity=50')
plt.show()


In [ ]:
#DBSCAN on tSNE (lower perplexity)
dbs3 = DBSCAN(min_samples=2, eps=0.3)
clust3 = dbs3.fit_predict(X_tsne2)

#Plotting
plt.figure(figsize=(10, 6))

scat4 = plt.scatter(X_tsne2[:, 0], X_tsne2[:, 1], c=clust3, cmap='viridis', marker='o', edgecolor='k', s=10)
plt.colorbar(scat4)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('DBSCAN Clusters after t-SNE perplexity=20')
plt.show()


In [ ]:
#DBSCAN on UMAP
dbs4 = DBSCAN(min_samples=2, eps=0.3)
clust4 = dbs4.fit_predict(X_umap)

#Plotting in 3D
fig,ax = plt.subplots(1,1, figsize=(10, 8))
ax = fig.add_subplot(projection='3d')

scat5 = ax.scatter(X_umap[:, 0], X_umap[:, 1],  X_umap[:,2], c=clust4, cmap='viridis', marker='o', edgecolor='k', s=10)
plt.colorbar(scat5, ax=ax)
ax.set_xlabel('PCA Component 1')
ax.set_ylabel('PCA Component 2')
ax.set_zlabel('PCA Component 3')
plt.title('DBSCAN Clusters after PCA')
plt.show()

### Performance

In [ ]:
from sklearn.metrics import homogeneity_score, completeness_score, silhouette_score
from collections import Counter

#Performance PCA
homogeneity = homogeneity_score(df_test, clusters)
completeness = completeness_score(df_test, clusters)
silhouette = silhouette_score(X_pca, clusters)

print(f'Homogeneity Score Kmeans on PCA: {homogeneity}')
print(f'Completeness Score Kmeans on PCA: {completeness}')
print(f'Silhouette Score Kmeans on PCA: {silhouette}')

# Determine contaminants in each cluster
contaminants = {}
for cluster_id in np.unique(clusters):
    if cluster_id == -1:  
        continue
    cluster_points = df_test[clusters == cluster_id]
    most_common_class, count = Counter(cluster_points).most_common(1)[0]
    contaminants[cluster_id] = len(cluster_points) - count

print(f'Contaminants in each cluster for Kmeans on PCA: {contaminants}')

In [ ]:
#Performance DBSCAN
homogeneity = homogeneity_score(df_test, clust)
completeness = completeness_score(df_test, clust)
silhouette = silhouette_score(X_pca, clust)

print(f'Homogeneity Score DBSCAN on PCA: {homogeneity}')
print(f'Completeness Score DBSCAN on PCA: {completeness}')
print(f'Silhouette Score DBSCAN on PCA: {silhouette}')

#Contaminants in each cluster
contaminants = {}
for cluster_id in np.unique(clust):
    if cluster_id == -1:  
        continue
    cluster_points = df_test[clust == cluster_id]
    most_common_class, count = Counter(cluster_points).most_common(1)[0]
    contaminants[cluster_id] = len(cluster_points) - count

print(f'Contaminants in each cluster for DBSCAN on PCA: {contaminants}')